# HW 2: Language Modeling

In this homework you will be building several varieties of language models.

## Goal

We ask that you construct the following models in PyTorch:

1. A trigram model with linear-interpolation. $$p(y_t | y_{1:t-1}) =  \alpha_1 p(y_t | y_{t-2}, y_{t-1}) + \alpha_2 p(y_t | y_{t-1}) + (1 - \alpha_1 - \alpha_2) p(y_t) $$
2. A neural network language model (consult *A Neural Probabilistic Language Model* http://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf)
3. An LSTM language model (consult *Recurrent Neural Network Regularization*, https://arxiv.org/pdf/1409.2329.pdf) 
4. Your own extensions to these models...


Consult the papers provided for hyperparameters.

 


## Setup

This notebook provides a working definition of the setup of the problem itself. You may construct your models inline or use an external setup (preferred) to build your system.

In [1]:
# Text text processing library
import torchtext
from torchtext.vocab import Vectors

The dataset we will use of this problem is known as the Penn Treebank (http://aclweb.org/anthology/J93-2004). It is the most famous dataset in NLP and includes a large set of different types of annotations. We will be using it here in a simple case as just a language modeling dataset.

To start, `torchtext` requires that we define a mapping from the raw text data to featurized indices. These fields make it easy to map back and forth between readable data and math, which helps for debugging.

In [2]:
# Our input $x$
TEXT = torchtext.data.Field()

Next we input our data. Here we will use the first 10k sentences of the standard PTB language modeling split, and tell it the fields.

In [3]:
# Data distributed with the assignment
train, val, test = torchtext.datasets.LanguageModelingDataset.splits(
    path=".", 
    train="train.txt", validation="valid.txt", test="valid.txt", text_field=TEXT)

The data format for language modeling is strange. We pretend the entire corpus is one long sentence.

In [4]:
print('len(train)', len(train))

len(train) 1


Here's the vocab itself. (This dataset has unk symbols already, but torchtext adds its own.)

In [5]:
TEXT.build_vocab(train)
print('len(TEXT.vocab)', len(TEXT.vocab))

len(TEXT.vocab) 10001


When debugging you may want to use a smaller vocab size. This will run much faster.

In [6]:
if False:
    TEXT.build_vocab(train, max_size=1000)
    len(TEXT.vocab)

The batching is done in a strange way for language modeling. Each element of the batch consists of `bptt_len` words in order. This makes it easy to run recurrent models like RNNs. 

In [7]:
train_iter, val_iter, test_iter = torchtext.data.BPTTIterator.splits(
    (train, val, test), batch_size=10, device=-1, bptt_len=32, repeat=False)

Here's what these batches look like. Each is a string of length 32. Sentences are ended with a special `<eos>` token.

In [8]:
it = iter(train_iter)
batch = next(it) 
print("Size of text batch [max bptt length, batch size]", batch.text.size())
print("Second in batch", batch.text[:, 2])
print("Converted back to string: ", " ".join([TEXT.vocab.itos[i] for i in batch.text[:, 2].data]))

Size of text batch [max bptt length, batch size] torch.Size([32, 10])
Second in batch Variable containing:
    8
  202
   77
    5
  183
  561
 3837
   18
  975
  976
    7
  943
    5
  157
   78
 1571
  289
  645
    3
   30
  132
    0
   20
    2
  273
 7821
   17
    9
  117
 2815
  969
    6
[torch.LongTensor of size 32]

Converted back to string:  in part because of buy programs generated by stock-index arbitrage a form of program trading involving futures contracts <eos> but interest <unk> as the day wore on and investors looked ahead to


The next batch will be the continuation of the previous. This is helpful for running recurrent neural networks where you remember the current state when transitioning.

In [9]:
batch = next(it)
print("Converted back to string: ", " ".join([TEXT.vocab.itos[i] for i in batch.text[:, 2].data]))

Converted back to string:  the release later this week of two important economic reports <eos> the first is wednesday 's survey of purchasing managers considered a good indicator of how the nation 's manufacturing sector fared


There are no separate labels. But you can just use an offset `batch.text[1:]` to get the next word.

## Assignment

Now it is your turn to build the models described at the top of the assignment. 

Using the data given by this iterator, you should construct 3 different torch models that take in batch.text and produce a distribution over the next word. 

When a model is trained, use the following test function to produce predictions, and then upload to the kaggle competition: https://www.kaggle.com/c/cs287-hw2-s18

For the final Kaggle test, we will have you do a next word prediction task. We will provide a 10 word prefix of sentences, and it is your job to predict 10 possible next word candidates

In [10]:
!head input.txt

but while the new york stock exchange did n't fall ___
some circuit breakers installed after the october N crash failed ___
the N stock specialist firms on the big board floor ___
big investment banks refused to step up to the plate ___
heavy selling of standard & poor 's 500-stock index futures ___
seven big board stocks ual amr bankamerica walt disney capital ___
once again the specialists were not able to handle the ___
<unk> james <unk> chairman of specialists henderson brothers inc. it ___
when the dollar is in a <unk> even central banks ___
speculators are calling for a degree of liquidity that is ___


As a sample Kaggle submission, let us build a simple unigram model.  

In [82]:
from collections import Counter
count = Counter()
for b in iter(train_iter):
    count.update(b.text.view(-1).data.tolist())
count[TEXT.vocab.stoi["<eos>"]] = 0
predictions = [TEXT.vocab.itos[i] for i, c in count.most_common(20)]
with open("sample.txt", "w") as fout: 
    print("id,word", file=fout)
    for i, l in enumerate(open("input.txt"), 1):
        print("%d,%s"%(i, " ".join(predictions)), file=fout)


In [86]:
!head sample.txt

id,word
1,the <unk> N of to a in and 's for that is $ said it on by at as from
2,the <unk> N of to a in and 's for that is $ said it on by at as from
3,the <unk> N of to a in and 's for that is $ said it on by at as from
4,the <unk> N of to a in and 's for that is $ said it on by at as from
5,the <unk> N of to a in and 's for that is $ said it on by at as from
6,the <unk> N of to a in and 's for that is $ said it on by at as from
7,the <unk> N of to a in and 's for that is $ said it on by at as from
8,the <unk> N of to a in and 's for that is $ said it on by at as from
9,the <unk> N of to a in and 's for that is $ said it on by at as from


The metric we are using is mean average precision of your 20-best list. 

$$MAP@20 = \frac{1}{|D|} \sum_{u=1}^{|D|} \sum_{k=1}^{20} Precision(u, 1:k)$$

Ideally we would use log-likelihood or ppl as discussed in class, but this is the best Kaggle gives us. This takes into account whether you got the right answer and how highly you ranked it. 

In particular, we ask that you do not game this metric. Please submit *exactly 20* unique predictions for each example.


As always you should put up a 5-6 page write-up following the template provided in the repository:  https://github.com/harvard-ml-courses/cs287-s18/blob/master/template/